# Парсинг и подготовка данных
Большая и слабоструктурированная тетрадка с парсингом данных на разных этапах работы

In [ ]:
import re
import random
import pymorphy2
import pandas as pd
import requests
from string import punctuation
from collections import Counter
from pprint import pprint
from nltk import ngrams

morph = pymorphy2.MorphAnalyzer()

In [2]:
TOKEN = TOKEN  # insert yours here
post_step = 100
comment_step = 100

## Первый этап: данные для проверки гипотезы

Критерии для пабликов: 
* много активности = ответов на комментарии, дискуссий
* комментарии открыты
* автоматическая модерация по ключевым словам
* отсутствие ручной модерации (чтобы комментарии с замаскированными ругательствами не удаляли)
* спорная тематика - чтобы был повод для агрессии. Это одновременно и необходимость, и огранчение

Паблики:
* лентач (новости, политика)
* медуза (новости, политика)
* дождь (новости, политика)  
мдк, двач, раша тудей, первый канал, риа новости, икволити, новая газета, эхо москвы, госдума и несколько других не подошли - либо нет автоматической модерации, либо есть ручная, либо нет второй стороны для споров, либо нет комментариев вообще  


Ограничение на тематику:
* в развлекательных пабликах часто нет фильтра на мат
* в тематических пабликах нет повода для споров и применения грубой лексики
* в пабликах на спорные чувствительные темы (например, правозащита или активизм) часто ручная модерация
* в провластных пабликах нет противоположного мнения
Видимо, подходят только условно оппозиционные паблики, куда часто приходят ругаться с "противоположной" стороны

In [3]:
meduza_id = -76982440
lentach_id = -29534144
dozhd_id = -17568841

In [20]:
def get_post_ids(group_id, n_posts):
    """
    collects given number of post ids
    """
    post_ids = []
    for off in range(0, n_posts, post_step): 
        data = requests.get(
            'https://api.vk.com/method/wall.get', 
            params={
                "owner_id": group_id,
                "count": post_step,
                "v":"5.131",
                "offset": off,
                "access_token": TOKEN
            }
        ).json()
        for item in data['response']['items']:
            post_ids.append(item['id'])
    return post_ids

In [5]:
def get_comment_texts(group_id, post_id_list, threshold):
    """
    collects comments from given posts, stops at a threshold
    """
    comments = []
    for post_id in post_id_list:
        for off in range(0, 1000, comment_step):
            data = requests.get(
                'https://api.vk.com/method/wall.getComments', 
                params={
                    "owner_id": group_id,
                    "post_id": post_id,
                    "preview_length": 0,
                    "thread_items_count": 10, # max 10
                    "count": comment_step, # def 10, max 100
                    "v":"5.131",
                    "offset": off,
                    "access_token": TOKEN
                }
            ).json()
            # pprint(data)
            for item in data['response']['items']:
                comments.append(item['text'])
                if item['thread']['count'] > 0:
                    for reply_comment in item['thread']['items']:
                        comments.append(reply_comment['text'])
        if len(comments) > threshold:
            break
    return comments

In [6]:
groups = [meduza_id, lentach_id, dozhd_id]

def parse(n_posts, threshold):
    """
    runs parsing and writes data to corpus dictionary
    """
    comment_corpus = {'source': ['медуза']*threshold + ['лентач']*threshold + ['дождь']*threshold,
                      'comments': []}

    for group_id in groups:
        post_ids = get_post_ids(group_id, n_posts)
        post_comments = get_comment_texts(group_id, post_ids, threshold)
        comment_corpus['comments'].extend(post_comments[:threshold])

    return comment_corpus

In [ ]:
# corpus = parse(100, 1000)

In [43]:
import pandas as pd

corpus_df = pd.DataFrame(comment_corpus)
corpus_df

,source,comments
0,медуза,Григорий Спел)))
1,медуза,Только новую серию Намедни закончил смотреть и...
2,медуза,А будет ли фильм Русские русские?
3,медуза,"[id415480453|Олег], есть же, ""Цвет нации""."
4,медуза,Что бы ни делал Леонид Геннадьевич - шедеврально.
...,...,...
2995,дождь,"[id138280294|Антон], не повторяйся бот"
2996,дождь,"[id49464409|Анатолий], Это констатация факта...."
2997,дождь,"[id138280294|Антон], утром методичку новую дали"
2998,дождь,"[id49464409|Анатолий], ага, Байден все делает ..."


In [44]:
corpus_df.to_csv('vk_corpus.csv', encoding='utf-8', sep='\t')

## Новые данные для нейросети

**Варианты датасетов для мультитаск нейросети:**

* данные для двух задач - оба только мои по вк
    * токсичность (2874 токс+нетокс немаск + 4.2% (126 шт) токс+нетокс маск)
    * маскировка (1800 маск+немаск (900 детекшн и разметка и 900 аугментация))

* данные для двух задач - все имеющиеся по токсичности, только мой по маскировкам
    * токсичность (2874 токс+нетокс немаск + 4.2% (126 шт) токс+нетокс маск + все остальные маск+немаск)
    * маскировка (1800 маск+немаск (900 детекшн и разметка и 900 аугментация))    

### Парсинг новых комментов

In [8]:
def parse_one(group_id, n_posts, threshold):
    """
    parses comments for one group
    """
    post_ids = get_post_ids(group_id, n_posts)
    post_comments = get_comment_texts(group_id, post_ids, threshold)
    
    return post_comments

In [11]:
meduza_comments = parse_one(meduza_id, n_posts=400, threshold=14500)
print(len(meduza_comments))

6052


In [15]:
meduza_comments2 = parse_one(meduza_id, n_posts=400, threshold=14500)
print(len(meduza_comments2))

6362


In [16]:
all_meduza_comments = meduza_comments + meduza_comments2

In [18]:
with open('all_meduza_comments.txt', 'w', encoding='UTF-8') as f:
    for comment in all_meduza_comments:
        f.write(comment + '\n--\n')

In [21]:
lentach_comments = parse_one(lentach_id, n_posts=400, threshold=14500)
print(len(lentach_comments))

14835


In [22]:
with open('all_lentach_comments.txt', 'w', encoding='UTF-8') as f:
    for comment in lentach_comments:
        f.write(comment + '\n--\n')

In [86]:
dozhd_comments = parse_one(dozhd_id, n_posts=400, threshold=14500)
print(len(dozhd_comments))

14510


In [87]:
with open('all_dozhd_comments.txt', 'w', encoding='UTF-8') as f:
    for comment in dozhd_comments:
        f.write(comment + '\n--\n')

In [88]:
print(len(all_meduza_comments), len(lentach_comments))

12414 14835


In [89]:
# create corpus
new_comments = {'source': ['медуза']*len(all_meduza_comments) + \
                          ['лентач']*len(lentach_comments) + \
                          ['дождь']*len(dozhd_comments),
                'comments': []}
new_comments['comments'].extend(all_meduza_comments)
new_comments['comments'].extend(lentach_comments)
new_comments['comments'].extend(dozhd_comments)

second_sample = pd.DataFrame(new_comments)
second_sample.head(5)

,source,comments
0,медуза,"Да как же вы так заблокированы, коли в контакте чуть не чаще ФАН отмечаетесь?"
1,медуза,"[id283595911|Владимир], ФАН-это унылое пригожинское говно. Не нужно сравнивать его с Медузой! А Медузу мы и так читаем и видим."
2,медуза,"[id183748796|Сергей], я не оцениваю, я констатирую."
3,медуза,"[id283595911|Владимир], можно было это фуфло не упоминать. Уважающий себя человек это говно не читает."
4,медуза,"[id183748796|Сергей], да бог с вами, читайте что хотите!"


In [90]:
len(second_sample)

41759

In [91]:
# delete comments that were included in the first sample
first_sample = pd.read_csv('корпуса/vk_labeled.tsv', sep='\t')
clean_second_sample = second_sample[~second_sample.comments.isin(first_sample.comments)]
len(clean_second_sample) # approx. 40k

41665

In [34]:
pd.set_option('display.max_colwidth', None)

In [93]:
clean_second_sample = clean_second_sample.sample(len(clean_second_sample))

In [95]:
clean_second_sample.to_csv('vk_corpus_total.csv', encoding='utf-8', sep='\t')

### Детекция комментариев с возможными маскировками

In [708]:
vk_link = re.compile('^\[.*?|.*?\], ')
http_link = re.compile('https?[0-9a-zA-Z.:/]*')
digits = re.compile('^\d+ | \d+ | \d+$')
punct = re.compile('^[\W\d]+ | [\W\d]+ | [\W\d]+$')
nonletters = re.compile('[^а-яёА-ЯЁ ,]')
spec_letters = [re.compile(item) for item in ['йо', '^мл', 'цц', '^ип']]
emoji = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)

def check_comment(text, debug=False):
    """
    checks given comments for signs of present distortion 
    """
    # step 1: remove links
    if text.startswith('['):
        text = re.sub(vk_link, '', text)
    text = re.sub(http_link, '', text)
    # step 2: remove separate numbers
    text = re.sub(digits, ' ', text)
    # step 3: remove single punctuations
    text = re.sub(punct, ' ', text)
    # step 4: remove emojis
    text = re.sub(emoji, '', text)
    # step 5: rstrip some punctuation for remaining words (or there will be too many false positives)
    text = ' '.join([token.rstrip('!"\'),-.:;?»…”').lstrip('“«"\'') for token in text.split()])
    # step 6: remove dashes that are not used for distortions
    for dashed_word in ['-то', '-либо', '-нибудь', 'из-за', 'из-под', '-петербург', 'кое-', '-таки']:
        text = text.replace(dashed_word, '')
    # step 7: if [^а-яёА-ЯЁ ,] pattern is in the remaining string - it's probably a distortion example
    if bool(re.search(nonletters, text)):
        if debug:
            print(re.search(nonletters, text))
        return True
    # step six: if special letter pattern (йо, ^мл, цц, ^ип) is in -> True
    if any(bool(re.search(pattern, text)) for pattern in spec_letters):
        if debug:
            print([re.search(pattern, text) for pattern in spec_letters])
        return True
    # does not take into account distortions with punctuation on the right, commas, letter duplications or deletions
    return False

In [97]:
detected_distortions = clean_second_sample[clean_second_sample['comments'].apply(check_comment)]
print(len(detected_distortions))

9429


In [98]:
detected_distortions.sample(15)

,source,comments
6753,медуза,"Идея ""быть украинцем"" это примерно процентов на 20 результат украинизации австро-венгерской (например, с помощью концлагеря Телергоф и др.), и на оставшиеся 80 - советской."
165,медуза,"[id83411201|Анар], всё еще впереди! И колбаса дяди Вовы, которую он положил на будущее страны. Со скрепным вкусом птицы-мехобвалки и белкового стабилизатора ""свиная шкурка""."
22698,лентач,"[id24769898|Григорий], и как х0хлы скулят и как Зеля попрошайничает и своих на Азовстали бросает 💩"
22644,лентач,"Бантустан Украина гордо и красиво раскинулся на русских просторах УССР-СССР ,подаренных Киеву сифилитиком В Ульяновым в 1918-1922 гг. В 2014 г,после ""Революции Достоинства"" в Киеве, я предлагал друзьям-украинцам провести пограничное цивилизованное размежевание с РФ,как провели его Франция-Германия (референдумы,опросы,переселение,выплата компенсаций и т д).""Всё наше""-сказали украинцы. ""Не проглотите,обделаетесь кровавым поносом""-предупредил я БЫВШИХ братьев. ""За призывы к референдумам мы тебя отыщем и удавим""-пригрозили мне украинцы и отправились на танках покорять русский Донбасс.Украинцы сегодня умирают за Родину-Украину,не понимая, что её у них уже украли и они умирают за Новую Великую Хазарию. В Природе тоже так бывает, из локона гусеницы, инфицированной яйцами ос, весной вылетит не бабочка, о осиный рой."
33829,дождь,"Время обьявить ввп международным террористом# Можно и по-тихому, всё как он любит )))))"
10478,медуза,"Ник - реально сопливый холуеныш-пидорок, осваивающий профессию проститутки:)"
3816,медуза,"[id349297371|Алексей], зачем жители Донбасса призывали в 2014 году Путина ввести войска и начать войну?Не понимают последствия своих действий лишь те, кто недостаточно хорошо изучал историю, которая очень часто повторяется."
1299,медуза,"урок всем, выбирающим непрофессиональных президентов, будь то артист или кгб-шник! президент должен быть политологом, историком и дипломатом!"
32268,дождь,"[id617119893|Dan], такие как ты - да.✌"
19800,лентач,"[id555441796|Аркадий], лгунишка. https://www.theguardian.com/world/2022/apr/24/dozens-bucha-civilians-killed-flechettes-metal-darts-russian-artillery"


In [107]:
detected_distortions['comments'] = detected_distortions['comments'].str.replace('\n', ' ')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
detected_distortions.to_csv('vk_detected_distortions_TOBELABELED.csv', encoding='utf-8', sep='\t')

In [102]:
import re

with open('vk_detected_distortions_TOBELABELED.csv', encoding='utf-8') as f:
    text = f.read()
    text = text.replace(r'\n', ' ')
with open('vk_detected_distortions_TOBELABELED_excel.csv', 'w', encoding='utf-8') as f:
    f.write(text)

### Дополнительная проверка на латиницу
Для комментариев, которые были вручную размечены как не содержащие маскировки, необходимо проверить автоматически наличие латинских букв, визуально совпадающих с кириллическими

In [202]:
labeled = pd.read_csv('корпуса/labeled_distortions.tsv', encoding='UTF-8', sep='\t', index_col='Column1')

In [203]:
labeled = labeled.head(3289)

In [204]:
labeled['distortion'].value_counts()

0    2913
1     376
Name: distortion, dtype: int64

In [164]:
labeled.head(3)

,source,comment,distortion
Column1,,,
14616.0,лентач,"[id720973152|Сабир ]масульманин? Если да,то скажи мне где в Коране написано,что божно обращаться к богу с просьбой о смерти другого человека",0
2558.0,медуза,"[id717780224|Sam], он не может, он на задании. Ему ещё после смены все свои выcepы в отчёт надо будет собрать и сдать куратору. И пачку доносов приложить к отчёту. Иначе зарплату зажмут и премии лишат.",0
36108.0,дождь,"вот о каких он сейчас минусах вещает,всё сейчас реально поняли-руссеие теперь реально не сдаются",0


In [170]:
negative = labeled.loc[labeled['distortion']==0]

In [173]:
negative.index = negative.index.astype('int64')

In [175]:
negative.head(3)

,source,comment,distortion
Column1,,,
14616,лентач,"[id720973152|Сабир ]масульманин? Если да,то скажи мне где в Коране написано,что божно обращаться к богу с просьбой о смерти другого человека",0
2558,медуза,"[id717780224|Sam], он не может, он на задании. Ему ещё после смены все свои выcepы в отчёт надо будет собрать и сдать куратору. И пачку доносов приложить к отчёту. Иначе зарплату зажмут и премии лишат.",0
36108,дождь,"вот о каких он сейчас минусах вещает,всё сейчас реально поняли-руссеие теперь реально не сдаются",0


In [189]:
lat_and_cyr = re.compile('[a-zA-Z]+[а-яА-Я]+|[а-яА-Я]+[a-zA-Z]+')

def find_latin(text, debug=False):
    """
    looks for latin letters after stripping reply link
    """
    if text.startswith('['):
        text = re.sub(link, '', text)
    text = re.sub(http_link, '', text)
    if bool(re.search(lat_and_cyr, text)):
        if debug:
            print(comment)
            print(re.findall('[a-zA-Z]+', text))
        return True

In [190]:
for index, row in negative.iterrows():
    comment = row.comment
    if find_latin(comment, debug=True):
        print('\n')
        if input() == 1:
            negative.at[index,'Age'] = 1
        

[id717780224|Sam], он не может, он на задании.  Ему ещё после смены все свои выcepы в отчёт надо будет собрать и сдать куратору. И пачку доносов приложить к отчёту.  Иначе зарплату зажмут и премии лишат.
['cep']


1
ХристоZ ВоскреZ!!! Слава руZZкому миру. Победа будет Zа нами. ??????
['Z', 'Z', 'ZZ', 'Z']



Вести из Китая ближе Я вижу Начинается история та Сибири Китай раздаёт паспорта Пение Дзин и первый свисток Паспорта Китая держи Дальний Восток Защита подданных - первый этап Суживаются глаза мам, детей, пап Якутия, Эвенкия, Ямал Прирост Шина таки йоу не мал Япония не отстаёт Курилам Камчатке свой Док раздаёт Pашка сжата до внутримкадья Потом вообще до кремля Единой Планеты Сёстры и Братья Летит летит песня моя Единства чудесный пир Всё Великим Путём В Лад и Мир!)))))
['P']


1
[id44844688|Владимир], eбанутая, никто не запрещал эвакуацию. Другое дело, что не все хотели покидать дома. Украинцы ведь не думали, что в 21 веке им устроят многомесячную блокаду.
['e']


1
Не, мы сами норм


[id1470163|Евгений], найди себе наконец парня и прекрати истерить, недотрахaннaя сyчкa.)
['a', 'a', 'y', 'a']


1
[id936775|Михаил], когда хохлы сало роняют, у руssких появляется возможность покушать??
['ss']



Zадержки V развитии
['Z', 'V']



Поганaя мрaзь.
['a', 'a']


1
Круто! Горжусь преzидентом! НАТО нам не надо! Прорвёмся!!!
['z']



[id318410827|Roger], эта бичyxa в дырявых сапогах ходит
['yxa']


1
Королёв, ХOХЛOТРOЛЛЬ
['O', 'O', 'O']


1
[id87868142|Святослав], Мне нравится как чел из самой жoпнoй и грязной страны мира говорит кому то хрюкнуть ??
['o', 'o']


1
[id617658661|Podporuchik], а руZZкие фашисты конечно не при делах. Они же жертвы ситуации
['ZZ']



[id5290963|Дмитрiй],
['id', 'i']



[id20489266|Александр], сын шлюxи, не гавкай.
['x']


1
[id3391071|Дмитрий], нaxyй твоя жопа хороша, пидop тупой
['axy', 'op']


1
[id660475021|Olika], Z это зига. Зига-зага руZZичи.
['Z', 'ZZ']



? Открытый микpофон “Стpану – сдают? Защитникoв – уничтoжают? Что делать?”    ?24 апре


[id569409622|Уинстон], не нравится - пиздуйтe в не ксенофобную Россию обратно????
['e']


1
[id283595911|Владимир], престарелый сын шлюxи, молодость не вернуть, а мудрость не купить. В твоём случае только эвтаназия.)
['x']


1
[id627854|Сергей], иx ищет интерпол
['x']



[id223651410|Яна], мне плевать русские они или якуты - я лишь констатировал факт. Если ты узкоглазaя хyитa, то вопросов нет, у каждого своя внешность. Не стесняйся указывать национальность нацистов, ну.)
['a', 'y', 'a']


1
Тот факт, что мне удобнее здесь смотреть мемы чем где-либо не отменяет тот факт, что это контора пидapacoв.
['apaco']


1
[id595984371|Денис], слейся нaxyй, если не осознаешь настолько базовых вещей.
['axy']


1
[id377141647|Иван], хрюкни ещё, Zигун
['Z']



Случайно прочитав ее фамилию как "Нaeбyллина" один раз, после уже не возможно читать ее правильно.
['ae', 'y']


1
[id629099314|Алла], это с какого такие выводы? Т.е. защита мирного населения от обстрелов укрокарателей, это "преступления против

1
Вы так говорите, словно вы бы не запретили акции "Zа мир" если бы у вас была такая возможность.
['Z']



Я СПИСОК КОРАБЛЕЙ ПРОЧЕЛ ДО СЕРЕДИНЫ  Утро было пасмурным и хмурым  В даль, в туман, подальше от земли  От причала мировой культуры  Уходили в море корабли.  Первым, ощетинив грозно пушки,  Ржавой чешуёй как жар горя,  Гордо шёл авианосец "Пушкин",  В трюмах – тридцать три богатыря.  Следом, на войну и мир готовый,  С сизой дыма бородой густой  Крейсер плыл огромный, но не новый –  "Граф Лев Николаевич Толстой",  А за ним кильватерной полоской –  Три эсминца, всем внушая страх –  "Бродский", "Маяковский" и "Чайковский"  Медленно плелись на всех парах.  Чайки на лету лишались чувства,  Мухи отделялись от котлет,  Видя баржи "Русское Искусство"  "Достоевский" "Горький" и "Балет".  В родину и скрепы крепко веря,  Оркестровый загрузив десант,  Шёл за ними "Гергиев Валерий",  На носу неся гвардейский бант.  В каждый порт входившие без страха,  Ласканные славой и рублём  Посланы по напр

1
[id410299385|Валодя], пошел нaxyй, стрелочник мамкин
['axy']


1
[id79288090|Андрей], ты реально веришь в эту xyйню что написал?
['xy']


1
[id51326742|Виктория], мирные люди жили мирно, пока на них не напал легион орков убивая и насилуя женщин и детей. Этот 10 летний мальчик остался сиротой. Его отца убили на глазах, а мать насиловали 2 дня до смерти руzzкие ослоебители. Как ты думаешь, какая у него мечта?
['zz']



Если из России уедут все адекватные люди, то останутся только РУССКИЕ и к ним поедит ZOРО??????
['ZO']



[id44844688|Владимир], обезьяна, Шольц не равно Германия, германские политики и общество иного мнения (сейчас идут разговоры об импичменте), а его лживaя позиция свидетельствует о возможном наличии личных целей, которые он преследует.
['a']



https://youtu.be/1dOUqwQc8OE  Чувак на канале Латыниной детально разложил лживyю политику Байдена в отношении Украины ???????
['y']



Галктики освободим, нет наZхцизму. Первая ракета взлетит при помощи моего жопогорения
['Z']



[id283595911|Владимир], стаpaя дeбилкa, без мата в доме престарелых.
['pa', 'e', 'a']


1
[id703303630|Андрiйко], рогуль, у тобе личка открыта чи шо, гаваець тупiй? Западные посольства перебрались во Львив потому что Западенщина будет защищена от потока рогулей - одичалые недоэвропэйци в виде беженцев Европе ни трэба... ??????
['i']



[id355287|Александр], естественно. Я же патриот и родину люблю, а не мacтурбирую на плешивого путлера, который уничтожает Россию, как некоторые тут в камментах.
['ac']


1
Слишком дeгенеративная тема для обсуждения.
['e']


1
[id512734700|Вячеслав],пошел наxyй отсюда
['id', 'xy']


1
[id20359098|White], пошла нaxyй, сельская казашка.)
['axy']


1
[id25105430|Максим], каклошвайн, пошел нaxyй
['axy']


1
раньше я был против цензуры, но изучив матчасть того как устроена свобода слова я понял что советская цензура была чуть ли не совершенством в плане фильтрации гoвнa.  Ее не надо было ломать, только совершенствовать.
['o', 'a']


1
[id270735056|Кирилл], во

1
Переименовалась на Еwa
['wa']



[id554098782|Артём], то есть если не сказал на стенд-апе то, о чём не хочет - трусливый чухaн?)
['a']


1
[id675126538|Abu], дyp, это видео после…
['yp']


1
[id291311787|МихУил], я тебе, 6лядвa хохлорылая, лично череп проcтpeлю, мне нужен только приказ,  червь ты хохлодырый????????????????
['a', 'c', 'pe']


1
[id270484805|Александр], котлеты по-руzки
['z']



[id21875698|Игорь], плесень у тебя на eблетe, овoщ.
['e', 'e', 'o']


1
[id138897333|Ромуальд], рыжий дeбик - сын вокзальной проститyтки, её любовь к дешёвой синтетике в период беременности имела последствия ???????
['e', 'y']


1
Когда через пару дней большой группировке всу под Донецком в рот напихают, кaклы и либeoда на своих горящих сpaльниках наверное в стратосферу  улетят ??
['a', 'eo', 'pa']


1
В качестве голубя подойдёт любой поцреот с пропеллером в задницe ??
['e']


1
[id223651410|Яна], дочь шлюxи, тем не менее ты судишь о моем происхождении или представлениях лишь на основе славянск

1
[id248938032|Марина], если их нaxyй не послали, а они добровольно съехали, то смею предположить, что не такие уж были светлые умы.
['axy']


1
Как отреагирует на это вaтники и кремлеботы?
['a']


1
[id4784072|Даур], им не мешает заламывать детей на митингах и насиловaть дубинками - эти преступления не против человечности, а другое?)
['a']



[id267020174|Viktor], ты либо крестик сними, либо фaшиZтом не будь  UPD: перестал,  по крайней мере, а интернете??
['a', 'Z', 'UPD']



[id653973289|Алексей], я roвном в отличии от тебя не питаюсь
['ro']


1
[id170542218|Nikolaj], слышь хохол тупой, краснов воевал на стороне Гитлера как и дивизия сс галичина которая вас истребляла. Это факт. И они для вас герои? Вы вообще тут конченые нaxyй?
['axy']


1
[id666676778|Туманный], тыж, пидpилa тут звенел об отсутствии экономики в РФ, падаль ты натужная. Методичку новую повезли, трёколо болотное?
['p', 'a']


1
Пополнение Zомби взращивают
['Z']



Контора пидapacoв получила по заслугам
['apaco']


1


In [192]:
negative.head(50)

,source,comment,distortion
Column1,,,
14616,лентач,"[id720973152|Сабир ]масульманин? Если да,то скажи мне где в Коране написано,что божно обращаться к богу с просьбой о смерти другого человека",0
2558,медуза,"[id717780224|Sam], он не может, он на задании. Ему ещё после смены все свои выcepы в отчёт надо будет собрать и сдать куратору. И пачку доносов приложить к отчёту. Иначе зарплату зажмут и премии лишат.",0
36108,дождь,"вот о каких он сейчас минусах вещает,всё сейчас реально поняли-руссеие теперь реально не сдаются",0
22935,лентач,ХристоZ ВоскреZ!!! Слава руZZкому миру. Победа будет Zа нами. ??????,0
8124,медуза,"[id278007933|Igor], он, вроде, не с чекистами договаривался, а с юристами перед т.н. выборами, чтобы отцепились. От России денег нет, геморроя много, Дуров россиянам помогает, и он же у и россиян всегда виноват и у российского правительства виноват. Он, похоже, просто всех послал, и, как теперь выясняется, правильно сделал.",0
38584,дождь,"Интересно будет послушать рассказ, захваченного участника украинской диверсионно-разведовательной группы. И как от него открестятся украинские власти.",0
34084,дождь,"Вести из Китая ближе Я вижу Начинается история та Сибири Китай раздаёт паспорта Пение Дзин и первый свисток Паспорта Китая держи Дальний Восток Защита подданных - первый этап Суживаются глаза мам, детей, пап Якутия, Эвенкия, Ямал Прирост Шина таки йоу не мал Япония не отстаёт Курилам Камчатке свой Док раздаёт Pашка сжата до внутримкадья Потом вообще до кремля Единой Планеты Сёстры и Братья Летит летит песня моя Единства чудесный пир Всё Великим Путём В Лад и Мир!)))))",0
29979,дождь,????,0
396,медуза,"[id44844688|Владимир], eбанутая, никто не запрещал эвакуацию. Другое дело, что не все хотели покидать дома. Украинцы ведь не думали, что в 21 веке им устроят многомесячную блокаду.",0


In [194]:
with open('logs.txt', encoding='UTF-8') as f:
    lines = f.readlines()

In [211]:
keys = [line.rstrip('\n') for line in lines[::5]]
vals = list(map(lambda x: 1 if x == '1\n' else 0, lines[4::5]))

In [227]:
vals[-1]

0

In [218]:
for k, v in zip(keys, vals):
    labeled.loc[labeled['comment'] == k, 'distortion'] = v

In [233]:
sent = 'Контора пидapacoв получила по заслугам'
labeled.loc[labeled['comment'] == sent, 'distortion'] = 1

In [235]:
labeled['distortion'].value_counts()

0    2716
1     573
Name: distortion, dtype: int64

In [236]:
labeled.to_csv('fully_labeled_distortions.tsv', encoding='utf-8', sep='\t')

### Аугментация маскированного класса

In [242]:
# remove labeled as positive for distortions from total sample
labeled_dist = labeled.loc[labeled['distortion'] == 1]
source_data = clean_second_sample[~clean_second_sample.comments.isin(labeled_dist.comment)]

In [273]:
test_data = pd.read_csv('корпуса/vk_labeled.tsv', encoding='UTF-8', sep='\t')
test_data.head(5)

,comment_id,source,comments,distortion,toxicity,corrected,Столбец1
0,0,медуза,Григорий Спел))),0,0,NaN,NaN
1,1,медуза,Только новую серию Намедни закончил смотреть и тут это. Каеф.,0,0,NaN,NaN
2,2,медуза,А будет ли фильм Русские русские?,0,0,NaN,NaN
3,3,медуза,"[id415480453|Олег], есть же, ""Цвет нации"".",0,0,NaN,NaN
4,4,медуза,Что бы ни делал Леонид Геннадьевич - шедеврально.,0,0,NaN,NaN


In [275]:
test_negative = test_data.loc[test_data['distortion'] == 0]

In [284]:
for index, row in test_negative.iterrows():
    comment = row.comments
    if not isinstance(comment, float): # skip empty comments (nan seen as float)
        if find_latin(comment, debug=True):
            print(index)
            print('\n')
            if input() == 1:
                test_negative.at[index,'distortion'] = 1

Кадыров все таки красавчик, вaту нагнул
['a']
67


1
[id248753181|Людмила], про бояться смешно, конечно, нacpaльного и весь его штаб уничтожили менее чем за полгода, сам он уже вряд ли когда-нибудь выйдет. От его сторонников никаких протестов, митингов и т.д., слили вы вождя своего, oчкoшники, и как вас после этого серьезно воспринимать?))
['acpa', 'o', 'o']
153


1
Мepзкая ситуация, видимо стоит ожидать какой-то провокации на этой или следующей неделе. Думаю любому здравомыслящему человеку понятно, что России эта война вообще не нужна, она ничего не получит от вторжения, только убытки. А вот штатам как раз эта война очень даже на руку - новые зверские санкции, окончательная изоляция России, возможность прибрать к рукам европейский рынок и создать образ нового мирового врага. Такие твapи они, конечно, ну надеюсь хоть мoзгов хватит третью мировую не начинать
['ep', 'ap', 'o']
212


1
[id3161208|Андрей], детский попытка перевода стрелок, зоогeй????.
['e']
331


1
[id162364532|Моника], по

In [288]:
with open('test_logs.txt', encoding='UTF-8') as f:
    lines = f.readlines()

In [289]:
keys = [line.rstrip('\n') for line in lines[::6]]
vals = list(map(lambda x: 1 if x == '1\n' else 0, lines[5::6]))

In [292]:
vals[-1] = 1
vals[-2:]

[1, 1]

In [293]:
for k, v in zip(keys, vals):
    test_data.loc[test_data['comments'] == k, 'distortion'] = v

In [294]:
test_data['distortion'].value_counts()

0    2874
1     126
Name: distortion, dtype: int64

In [295]:
test_data.to_csv('vk_labeled_addedLatinDist.tsv', encoding='utf-8', sep='\t')

### Функции замены

In [580]:
cyr_to_lat = {'е': 'e', 'т': 't', 'у': 'y',
              'и': 'u', 'о': 'o', 'р': 'p',
              'а': 'a', 'н': 'h', 'к': 'k',
              'х': 'x', 'с': 'c', 'в': 'b',
              'м': 'm', 'п': 'n'}

In [595]:
cyr_to_cyr = {'е': 'и',
            # 'еб': 'ип',
            # 'бл': 'мл',
            # 'ов': 'ау',
              'б': 'п',
              'д': 'т',
              'х': 'к',
              'о': 'а',
              'з': 'с',
              'ж': 'ш',
              'в': 'у',
              'с': 'ц'}
            # 'тс': 'цц',
            # 'пи': '3,14'}

In [596]:
class NotApplicable(Exception):
    """
    raised when selected distortion method is not applicable to string
    """
    pass

In [597]:
def latin_letter(word): # P = 52/140
    """
    distortion with a latin letter
    """
    letters = list(word)
    if set(letters) & set(cyr_to_lat.keys()):
        while True:
            to_replace = random.choice(range(len(letters)))
            if letters[to_replace] in cyr_to_lat:
                break
        letters[to_replace] = cyr_to_lat[letters[to_replace]]
        return ''.join(letters)
    else:
        raise NotApplicable

def similar_letter(word): # P = 17+3(pi)/140
    """
    distortion with a similat cyrillic letter or 3,14
    """
    letters = list(word)
    if set(letters) & set(cyr_to_cyr.keys()):
        while True:
            to_replace = random.choice(range(len(letters)))
            if letters[to_replace] in cyr_to_cyr:
                break
        letters[to_replace] = cyr_to_cyr[letters[to_replace]]
        return ''.join(letters)
    else:
        raise NotApplicable

def fullstop(word): # P = 10/140
    """
    distortion with .
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    letters[to_replace] = '.'
    return ''.join(letters)

def two_dots(word): # P = 8/140
    """
    distortion with ..
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    letters[to_replace] = '..'
    return ''.join(letters)

def precent_asterisc_drop(word): # P = 7*3/140
    """
    distortion with %, * or dropped letter
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    replace_with = random.choice(['%', '*', ''])
    letters[to_replace] = replace_with
    return ''.join(letters)

def at(word): # P = 5/140
    """
    distortion with @
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    letters[to_replace] = '@'
    return ''.join(letters)

def threedots_number(word): # P = 4*2/140
    """
    distortion with ... or a digit
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    replace_with = random.choice([str(random.choice(range(10))), '...'])
    letters[to_replace] = replace_with
    return ''.join(letters)

def parenthesis(word): # P = 3/140
    """
    distortion with ) or ( 
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    replace_with = random.choice(['(', ')'])
    letters[to_replace] = replace_with
    return ''.join(letters)

def threeasteriscs_repetition(word): # P = 2*2/140
    """
    distortion with ***
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    replace_with = random.choice(['***', letters[to_replace]*2])
    letters[to_replace] = replace_with
    return ''.join(letters)

def other_ways(word): # P = 1*9/140
    """
    distortion with other ways
    """
    letters = list(word)
    to_replace = random.choice(range(len(letters)))
    replace_with = random.choice(['.....', '"', '#', ' ', '-', '....', ',', '()', '  '])
    if replace_with == '()':
        letters[to_replace] = f'({letters[to_replace]})'
        return ''.join(letters)
    elif replace_with == '  ':
        return ' '.join(letters)
    elif replace_with == ' ':
        letters.insert(to_replace, ' ')
        return ''.join(letters)
    else:
        letters[to_replace] = replace_with
        return ''.join(letters)

In [627]:
def distort_word(word):
    """
    takes a word, returns distorted version according to the distribution
    of methods of distortion
    """
    possible_functions = [(latin_letter,()), (similar_letter,()), (fullstop,()),
                          (two_dots,()), (precent_asterisc_drop,()), (at,()), 
                          (threedots_number,()), (parenthesis,()),
                          (threeasteriscs_repetition,()), (other_ways, ())]
    weights = [51, 20, 10, 8, 21, 5, 8, 3, 4, 9]
    while True:
        try:
            (func, args), = random.choices(possible_functions, weights=weights)
            distorted = func(word)
            break
        except NotApplicable:
            pass

    return distorted

Замена:

In [599]:
with open('bad_wordlist.txt', encoding='UTF-8') as f:
    bad_wordlist = [line.rstrip('\n') for line in f.readlines()]
lemmed_wordlist = [morph.parse(token.lower())[0].normal_form  for token in bad_wordlist]

In [635]:
def distort_comment(text):
    """
    takes a comment, returns its version where one of the present 'bad words' is distorted.
    if none are present, returns nothing
    """
    tokens = [token.strip(punctuation) for token in text.split()]
    lemmas = [morph.parse(token.lower())[0].normal_form  for token in tokens]
    if set(lemmas) & set(lemmed_wordlist):
        while True:
            to_replace = random.choice(range(len(tokens)))
            if lemmas[to_replace] in lemmed_wordlist:
                break
        return text.replace(tokens[to_replace], distort_word(tokens[to_replace]))

In [639]:
to_distort = source_data.sample(10000)

In [640]:
to_distort['automatically_distorted'] = to_distort['comments'].apply(distort_comment)

In [641]:
len(to_distort.loc[to_distort['automatically_distorted'].notnull()])

1317

In [644]:
to_distort.loc[to_distort['automatically_distorted'].notnull()][['source', 'automatically_distorted']].head(3)

,source,automatically_distorted
36687,дождь,"[id627634435|Александр], у вас день открытых дверей в дурке ? Кто начал обстрел ЛДНР ? Сколько можно убивать людей. Всё правильно сделал Путиh."
1851,медуза,Отказ стран от российских энергоносителей может сподвигнуть российскую власть на реализацию их внутри самой Россиu?\n\nИли это другое?
18999,лентач,"[id160933923|Валентин],да.Потому что,она сука нацисткая с ср.ного Казахстана людоедского,конеедского."


In [645]:
augmented_distortions = to_distort.loc[to_distort['automatically_distorted'].notnull()][['source', 'automatically_distorted']]

In [646]:
augmented_distortions['distortion'] = 1

In [647]:
augmented_distortions.head(2)

,source,automatically_distorted,distortion
36687,дождь,"[id627634435|Александр], у вас день открытых дверей в дурке ? Кто начал обстрел ЛДНР ? Сколько можно убивать людей. Всё правильно сделал Путиh.",1
1851,медуза,Отказ стран от российских энергоносителей может сподвигнуть российскую власть на реализацию их внутри самой Россиu?\n\nИли это другое?,1


In [648]:
augmented_distortions.to_csv('augmented_distortions.tsv', encoding='utf-8', sep='\t')

### Выделить случайный семпл и разметить как НЕмаскированное
Необходимо 900 примеров с `distortion == 0`

In [659]:
# (all parsed minus lebaled as distorted) minus augmented
tolabel_notdistorted = source_data[~source_data.comments.isin(to_distort.loc[to_distort['automatically_distorted'].notnull()].comments)]
tolabel_notdistorted = tolabel_notdistorted.sample(1500)

In [660]:
tolabel_notdistorted['comments'] = tolabel_notdistorted['comments'].str.replace('\n', ' ')

In [661]:
tolabel_notdistorted.to_csv('to_label_for_notdistorted.tsv', encoding='utf-8', sep='\t')
# labeled file is 'labeled_notdistortions.tsv''

### Отделить часть маскировок на разметку токсичности для задачи 1

In [673]:
labeled_distortions = pd.read_csv('корпуса/fully_labeled_distortions.tsv', encoding='utf-8', sep='\t')
distorted_found = labeled_distortions.loc[labeled_distortions['distortion'] == 1]
len(distorted_found)

576

In [674]:
tolabel_for_tox = distorted_found.sample(126)
distorted_found = distorted_found[~distorted_found.comment.isin(tolabel_for_tox.comment)]
len(distorted_found), len(tolabel_for_tox)

(450, 126)

In [676]:
distorted_found.to_csv('found_distortions.tsv', encoding='utf-8', sep='\t')
tolabel_for_tox.to_csv('distortions_tolabel_for_tox.tsv', encoding='utf-8', sep='\t')

### Создание общих файлов с данными для задач

In [680]:
tox_pt1 = pd.read_csv('fully_labeled_test_corpus.tsv', encoding='UTF-8', sep='\t')
tox_pt1 = tox_pt1.loc[tox_pt1['distortion'] == 0][['comments', 'toxicity']]
tox_pt1.rename(columns = {'comments': 'comment'}, inplace = True)

tox_pt2 = pd.read_csv('distortions_labeled_for_tox.tsv', encoding='UTF-8', sep='\t')
tox_pt2 = tox_pt2[['comment', 'toxicity']]

In [682]:
toxicity_data = pd.concat([tox_pt1, tox_pt2], ignore_index=True).sample(3000)
toxicity_data.head(3)

,comment,toxicity
993,"Штош, пойду праздновать, влюблённых поздравлять",0
161,Честный человек ???????? вот как можно с серьёзным лицом такое говорить и не заржать??,0
668,"""Учения"" значит? Хитро придумано. Теперь уж точно никто не насторожится ))",0


In [ ]:
tox_pt1 = pd.read_csv('fully_labeled_test_corpus.tsv', encoding='UTF-8', sep='\t')
tox_pt1 = tox_pt1.loc[tox_pt1['distortion'] == 0][['comments', 'toxicity']]
tox_pt1.rename(columns = {'comments': 'comment'}, inplace = True)

tox_pt2 = pd.read_csv('distortions_labeled_for_tox.tsv', encoding='UTF-8', sep='\t')
tox_pt2 = tox_pt2[['comment', 'toxicity']]

In [696]:
dist_pt1 = pd.read_csv('found_distortions.tsv', encoding='UTF-8', sep='\t')
dist_pt1 = dist_pt1[['comment', 'distortion']]

dist_pt2 = pd.read_csv('augmented_distortions.tsv', encoding='UTF-8', sep='\t').sample(len(dist_pt1))
dist_pt2 = dist_pt2[['automatically_distorted', 'distortion']]
dist_pt2.rename(columns = {'automatically_distorted': 'comment'}, inplace = True)

dist_pt3 = pd.read_csv('labeled_notdistortions.tsv', encoding='UTF-8', sep='\t')
dist_pt3['distortion'] = 0
dist_pt3 = dist_pt3[['comments', 'distortion']]
dist_pt3.rename(columns = {'comments': 'comment'}, inplace = True)

In [698]:
distortion_data = pd.concat([dist_pt1, dist_pt2, dist_pt3], ignore_index=True)
distortion_data = distortion_data.sample(len(distortion_data))
distortion_data.head(3)

,comment,distortion
1358,"[id450436040|Евгений], ты давно новости открывал? мне ведь не нужно предъявлять тебе доказательства обратного?)",0
552,А до 2027 будут за рубли покупать?)) Да и хорошо 😂😂 Мне кажется Р***ссию это прекрасно устраивает)),1
44,Да кому он нафиг вcpaлся,1


In [699]:
toxicity_data.to_csv('toxicity_data.tsv', encoding='utf-8', sep='\t')
distortion_data.to_csv('distortion_data.tsv', encoding='utf-8', sep='\t')

In [700]:
distortion_data.distortion.value_counts(normalize=True)

0    0.5
1    0.5
Name: distortion, dtype: float64

In [707]:
from sklearn.model_selection import train_test_split

In [712]:
toxicity_data = pd.read_csv('toxicity_data.tsv', sep='\t', encoding='UTF-8')
distortion_data = pd.read_csv('distortion_data.tsv', sep='\t', encoding='UTF-8')

In [713]:
def remove_link(text):
    """
    removes author link form replies
    """
    return re.sub(vk_link, '', text)

In [720]:
toxicity_data.dropna(inplace=True)
toxicity_data['text'] = toxicity_data['text'].apply(remove_link)
distortion_data['text'] = distortion_data['text'].apply(remove_link)

tox_train, tox_val = train_test_split(toxicity_data[['text', 'target']],
                                      test_size=0.2, random_state=42)
dist_train, dist_val = train_test_split(distortion_data[['text', 'target']],
                                        test_size=0.2, random_state=42)

tox_train.reset_index(drop=True, inplace = True)
tox_train.rename(columns = {'text': 'doc'}, inplace = True)
tox_val.reset_index(drop=True, inplace = True)
tox_val.rename(columns = {'text': 'doc'}, inplace = True)
dist_train.reset_index(drop=True, inplace = True)
dist_train.rename(columns = {'text': 'doc'}, inplace = True)
dist_val.reset_index(drop=True, inplace=True)
dist_val.rename(columns = {'text': 'doc'}, inplace = True)

tox_train.to_csv('toxicity_train.tsv', encoding='utf-8', sep='\t', index_label='id')
tox_val.to_csv('toxicity_val.tsv', encoding='utf-8', sep='\t', index_label='id')
dist_train.to_csv('distortion_train.tsv', encoding='utf-8', sep='\t', index_label='id')
dist_val.to_csv('distortion_val.tsv', encoding='utf-8', sep='\t', index_label='id')

### Собрать в общий корпус все данные по токсичности

In [793]:
del toxicity_data['Unnamed: 0']

In [816]:
toxicity_data.rename(columns = {'text': 'doc'}, inplace = True)

In [817]:
kaggle = pd.read_csv('другие корпуса токсичности/kaggle_rltc.csv', encoding='UTF-8')
kaggle['toxic'] = kaggle['toxic'].astype(int)
kaggle.rename(columns={'comment': 'doc', 'toxic': 'target'}, inplace=True)
kaggle.head(3)

,doc,target
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1
2,Собаке - собачья смерть\n,1


In [818]:
with open('другие корпуса токсичности/ok_train.txt', encoding='UTF-8') as f:
    raw_ok = f.read()

In [819]:
raw_ok = raw_ok.replace('__label__NORMAL', '0')
raw_ok = re.sub('__label__\w+(\t__label__\w+)*', '1', raw_ok)

In [820]:
import io

In [821]:
ok_cup = pd.read_csv(io.StringIO((raw_ok)), sep='\t', names=['comment_id', 'target', 'doc'])
del ok_cup['comment_id']
ok_cup.head(3)

,target,doc
0,1,дворника надо тоже уничтожить!
1,0,"моя старшая неделю шипела, не принимала подкидыша, которого я принесла. китя такой славный, потерпите немного, мои теперь душа в душу живут, ваша примет попозже. а котенок необыкновенно красивый."
2,0,полностью с вами согласна!


In [822]:
russe_1 = pd.read_csv('другие корпуса токсичности/russe_dev.tsv', encoding='UTF-8', sep='\t')
russe_2 = pd.read_csv('другие корпуса токсичности/russe_train.tsv', encoding='UTF-8', sep='\t')
russe = pd.concat([russe_1, russe_2], ignore_index=True)
russe = russe[['toxic_comment']]
russe.rename(columns={'toxic_comment': 'doc'}, inplace=True)
russe['target'] = 1
russe.head(3)

,doc,target
0,пиздеж! температуры горения хватит чтобы её расплавить к херам..,1
1,а ты чмо там был.ты вообще служил.гандон,1
2,пиздабол ---- а сам где кормишься ?,1


In [823]:
print(len(kaggle), len(ok_cup), len(russe), len(toxicity_data))

14412 148775 7748 2897


### Sampling

In [824]:
toxicity_data['target'].value_counts()

0    2336
1     561
Name: target, dtype: int64

In [825]:
combined = pd.concat([toxicity_data, russe.sample(1775, random_state=42)], ignore_index=True)
print(combined['target'].value_counts())
combined = pd.concat([combined, kaggle.groupby('target').sample(n=1000, random_state=42)], ignore_index=True)
print(combined['target'].value_counts())
combined = pd.concat([combined, ok_cup.groupby('target').sample(n=1000, random_state=42)], ignore_index=True)
print(combined['target'].value_counts())

0    2336
1    2336
Name: target, dtype: int64
0    3336
1    3336
Name: target, dtype: int64
0    4336
1    4336
Name: target, dtype: int64


In [828]:
combined = combined.sample(len(combined), random_state=422)
combined.head(3)

,doc,target
6744,есть 3 балкона 195 60 15,0
3399,"Одноклассники суки! Напился,проблевал, проспал!(",1
3058,уёбишный артист и как человек уёбок,1


In [829]:
combined_train, combined_val = train_test_split(combined, test_size=0.2, random_state=42)

In [830]:
combined_train.to_csv('combined_train.tsv', sep='\t', encoding='UTF-8')
combined_val.to_csv('combined_val.tsv', sep='\t', encoding='UTF-8')